In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import re
import torch
import glob
import string
import math
import random

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import zipfile
import seaborn as sbr
import matplotlib.pyplot as plt 
import pandas as pd
import functools
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

SEED = 147
torch.manual_seed(SEED)
torch.__version__

'1.7.0+cu101'

In [ ]:
!pip install --upgrade pip
!pip install transformers
!pip install bpemb

from bpemb import BPEmb
from transformers import BertTokenizer, BertModel, AutoTokenizer, AdamW, BertForMaskedLM, Adafactor

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
total_df = pd.read_csv("/content/gdrive/My Drive/NL3.14/resources/geosentences.csv")

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("/content/gdrive/My Drive/NL3.14/resources/word2vec.model")

w2v = Word2Vec.load(fname)
# w2v = Word2Vec(sentences=train_data, sg=1, hs=1, size=300, workers=4, window=3, seed=SEED, sorted_vocab=1, min_count=3)

# w2v.save(fname)
# w2v.train(train_data, total_examples=len(train_data), epochs=5)

In [ ]:
final_df = pd.read_csv("/content/gdrive/My Drive/NL3.14/resources/train.csv")

In [ ]:
bert_train, bert_validation = train_test_split(total_df, test_size=0.05)
train, validation = train_test_split(final_df, test_size=0.05)

In [ ]:
train.head(50)

,x,y
1040202,"['და', 'კმარა', ',', 'თავი', 'დამანებე', ',']",მასარაკშ
127562,"['ამიტომ', 'დანიელი', ',', 'ემიგრანტეივით']",სატვირთო
390489,"['ეზოს', 'გავხედე', ',', 'ის', 'დიდი', 'იყო', ...",ერთ
597724,"['სამაგიეროდ', 'პატარა', 'საბაბიც', 'კი', 'საკ...",თავის
736530,"['ხოლო', 'ჩვენს', 'მიერ', 'დაფუძნებულ', 'სახელ...",მცველები
987744,"['იმ', 'თავითვე', ',', 'დავბადებულვარ', 'თუ', ...",გაგდებული
934493,"['ჟერვეზა', 'ამ', 'თავაზიანობის', 'საპასუხოდ',...",სახურავის
590549,"['ჩვენ', 'უბრალოდ', ',', 'თვალმოუშორებლივ', 'ვ...",ცეცხლის
338362,"['როგორც', 'ყოველთვის', ',', 'ჯიბეები', 'გამოტ...",მისთვის
1022993,"['შინ', 'მოსულს', 'ფეხსაცმელს', 'გავხდი', 'და']",დაჩოქილი


In [ ]:
embed_dim = 300
vocab_size = w2v.wv.vectors.shape[0]
max_seq_len = 32
batch_size = 32

In [ ]:
class DatasetTrain(torch.utils.data.Dataset):
  def __init__(self, txts):
        self.txts = txts

  def __len__(self):
        return len(self.txts)

  def __getitem__(self, index):
        sentence = self.txts[index]
        splits = sentence.split(' ')
        chosenWords = random.choices(splits, k = len(splits) // 10 + 1)
        for idx, word in enumerate(splits):
          if word in chosenWords:
            splits[idx] = '[MASK]'
  
        embedX = tokenizer(' '.join(splits),  padding='max_length', return_tensors='pt', max_length=max_seq_len, truncation=True)
        embedY = tokenizer(sentence, padding='max_length', return_tensors='pt', max_length=max_seq_len, truncation=True)
      
        return embedX['input_ids'].to(device),embedX['token_type_ids'].to(device), embedX['attention_mask'].to(device), embedY['input_ids'].to(device)

In [ ]:
# Parameters
train_params = {'batch_size': batch_size,
          'shuffle': True
         }
val_params = {'batch_size': batch_size
         }

# Dataloaders for train, validation and test
bert_training_set = DatasetTrain(bert_train['sentences'].tolist())
bert_training_generator = torch.utils.data.DataLoader(bert_training_set, **train_params)

bert_validation_set = DatasetTrain(bert_validation['sentences'].tolist())
bert_validation_generator = torch.utils.data.DataLoader(bert_validation_set, **val_params)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased').to(device)
model.load_state_dict(torch.load('/content/gdrive/My Drive/NL3.14/resources/bert_model'))
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

#Calculate loss on validation data
def bert_valid_loss(model, dl):
  #Switch model to evaluation mode and then back to train
  model.eval()
  with torch.no_grad(): # tells Pytorch not to store values of intermediate computations for backward pass because we not gonna need gradients.
    loss = 0
    batches = math.ceil(len(dl) / batch_size)
    for x_input_ids, x_token_type_ids, x_attention_mask, y_input_ids in bert_validation_generator:
        output = model(input_ids=x_input_ids.squeeze(1),
                     token_type_ids=x_token_type_ids.squeeze(1),
                     attention_mask=x_attention_mask.squeeze(1),
                     labels=y_input_ids,
                    )
        loss += output.loss.item()
  model.train()
  return loss / batches

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# from datetime import datetime
# # Switch model to train mode
# model.train()

# validation_loss = None
# for epoch in range(1, 2):
#     epoch_loss = 0
#     epoch_acc = 0
#     epoch_f_score = 0
#     batches = math.ceil(len(train) / batch_size)
#     print(batches, "batches")
#     t = datetime.now()
#     i = 0
#     for x_input_ids, x_token_type_ids, x_attention_mask, y_input_ids in bert_training_generator:
        
#         # if we don't do this, pytorch will accumulate gradients (summation) from previous backprop to next backprop and so on...
#         # this behaviour is useful when we can't fit many samples in memory but we need to compute gradient on large batch. In this case, we simply accumulate gradient and do backprop
#         # after enough samples has been seen.
#         optimizer.zero_grad()
#         #forward
#         # Model calculates loss and also outputs classification scores, which need to go through softmax later
#         output = model(input_ids=x_input_ids.squeeze(1),
#                      token_type_ids=x_token_type_ids.squeeze(1),
#                      attention_mask=x_attention_mask.squeeze(1),
#                      labels=y_input_ids,
#                     )
#         # print(output.loss.item())
#         #calculate accuracy
#         # acc = calculate_accuracy(log, y)

#         #Back propagation
#         output.loss.backward()
        
#         #Gradient step
#         optimizer.step()

#         epoch_loss += output.loss.item()
#         # epoch_acc += acc.item()
        
#         if (i + 1) % 1000 == 0:
#             print(datetime.now() - t)
#             dev_loss = bert_valid_loss(model, bert_validation)
#             if validation_loss is None or validation_loss > dev_loss:
#                 validation_loss = dev_loss
#                 # Save best model
#                 torch.save(model.state_dict(), '/content/gdrive/My Drive/NL3.14/resources/bert_model') 
#             print(f'Epoch {epoch} batch {i} | Avg Train Loss: {epoch_loss/(i + 1):.6f} | Current Dev Loss {dev_loss:.6f} | Minimal Dev Loss {validation_loss:.6f}')
#         i+=1
#     print(f'Epoch {epoch} | Avg Train Loss: {epoch_loss/batches:.6f} ')  

In [ ]:
batch_size = 256

class PredictionDatasetTrain(torch.utils.data.Dataset):
  def __init__(self, x, y):
        self.x = x
        self.y = y

  def __len__(self):
        return len(self.y)

  def __getitem__(self, index):
        def toEmbed(word):
          try:
            return w2v.wv.vocab[word].index
          except:
            try:
              return w2v.wv.vocab[w2v.wv.most_similar(word)[0][0]].index
            except:
              return random.randint(0, len(w2v.wv.vectors) - 1)
        x = [toEmbed(i) for i in self.x[index]]
        x = (x + [0]*max_seq_len)[:max_seq_len]
        # y = torch.LongTensor([w2v.wv.vocab[self.y[index]].index])
        y = nn.functional.one_hot(torch.LongTensor([w2v.wv.vocab[self.y[index]].index]), len(w2v.wv.vocab))
        return torch.LongTensor(x).to(device), y.to(device).float()

# Dataloaders for train, validation
next_training_set = PredictionDatasetTrain(train['x'].tolist(), train['y'].tolist())
next_training_generator = torch.utils.data.DataLoader(next_training_set, **train_params)

next_validation_set = PredictionDatasetTrain(validation['x'].tolist(), validation['y'].tolist())
next_validation_generator = torch.utils.data.DataLoader(next_validation_set, **train_params)

In [ ]:
class PredictionModel(nn.Module):
    def __init__(self, emb_dim, hid_dim, vocab_size):
        super().__init__()
        
        self.hidden_dim = hid_dim
        self.emb_dim = embed_dim
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(w2v.wv.vectors))

        self.lstm = nn.LSTM(self.emb_dim, self.hidden_dim, dropout=0.2, num_layers=2, bidirectional=True, batch_first=True)
        
        # self.lstm2 = nn.LSTM(self.hidden_dim * 4, self.hidden_dim * 2, batch_first=True)
    
        #last classifier is linear and we don't need sigmoid since loss function that we are going to use does apply sigmoid
        self.classifier = nn.Linear(4 * hid_dim, vocab_size)
              
    def forward(self, src):
        #src = [batch size, src len, emb dim]
        #embedded = [batch size, src len, emb dim]
        embedded = self.embedding(src)
        _, (hidden1, _) = self.lstm(embedded)
        # print(hidden1.shape)
        hidden1 = hidden1.reshape(-1, 4*self.hidden_dim) # A single dimension may be -1, in which case it’s inferred from the remaining dimensions and the number of elements in hs.
        # hidden1 = hidden1.unsqueeze(1)
        # print(hidden1.shape)
        # _, (hidden2, _) = self.lstm2(hidden1)

        #hidden = [1, batch size, hid dim]
        #cell = [1, batch size, hid dim]
        # hidden2 = hidden2.reshape(2*self.hidden_dim) # A single dimension may be -1, in which case it’s inferred from the remaining dimensions and the number of elements in hs.
        #outputs are always from the top hidden layer
        
        # print(hidden2.shape)
        #prediction = [batch size, 1]
        return self.classifier(hidden1)
        
pred_model = PredictionModel(embed_dim, 128, vocab_size).to(device)
pred_model.load_state_dict(torch.load('/content/gdrive/My Drive/NL3.14/resources/prediction_model'))
pred_model

PredictionModel(
  (embedding): Embedding(145338, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (classifier): Linear(in_features=512, out_features=145338, bias=True)
)

In [ ]:
# criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(pred_model.parameters(), lr=1e-4, eps=1e-7)
criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(pred_model.parameters(), lr=1e-4, weight_decay=1e-4)

#Calculate loss on validation data
def valid_loss(model, dl):
  #Switch model to evaluation mode and then back to train
  model.eval()
  loss = 0
  with torch.no_grad(): # tells Pytorch not to store values of intermediate computations for backward pass because we not gonna need gradients.
    batches = math.ceil(len(dl) / batch_size)
    for x, y in next_validation_generator:
        preds = model(x)
        loss += criterion(preds.squeeze(0), y.squeeze(1)).item()
  model.train()
  return loss / batches

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, bidirectional = True):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.bidirectional = bidirectional
    
    self.lstm = nn.LSTM(input_size, hidden_size, bidirectional = bidirectional)
  
  def forward(self, inputs, hidden):
    
    output, hidden = self.lstm(inputs.view(1, 1, self.input_size), hidden)
    return output, hidden
    
  def init_hidden(self):
    return (torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size),
      torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size))
    
class AttentionDecoder(nn.Module):
  
  def __init__(self, hidden_size, output_size, vocab_size):
    super(AttentionDecoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    
    self.attn = nn.Linear(hidden_size + output_size, 1)
    self.lstm = nn.LSTM(hidden_size + vocab_size, output_size) #if we are using embedding hidden_size should be added with embedding of vocab size
    self.final = nn.Linear(output_size, vocab_size)
  
  def init_hidden(self):
    return (torch.zeros(1, 1, self.output_size),
      torch.zeros(1, 1, self.output_size))
  
  def forward(self, decoder_hidden, encoder_outputs, input):
    
    weights = []
    for i in range(len(encoder_outputs)):
      print(decoder_hidden[0][0].shape)
      print(encoder_outputs[0].shape)
      weights.append(self.attn(torch.cat((decoder_hidden[0][0], 
                                          encoder_outputs[i]), dim = 1)))
    normalized_weights = F.softmax(torch.cat(weights, 1), 1)
    
    attn_applied = torch.bmm(normalized_weights.unsqueeze(1),
                             encoder_outputs.view(1, -1, self.hidden_size))
    
    input_lstm = torch.cat((attn_applied[0], input[0]), dim = 1) #if we are using embedding, use embedding of input here instead
    
    output, hidden = self.lstm(input_lstm.unsqueeze(0), decoder_hidden)
    
    output = self.final(output[0])
    
    return output, hidden, normalized_weights

In [ ]:
from datetime import datetime
# Switch model to train mode
pred_model.train()

validation_loss = None
for epoch in range(1, 5):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f_score = 0
    batches = math.ceil(len(train) / batch_size)
    print(batches, "batches")
    t = datetime.now()
    i = 0
    for x, y in next_training_generator:
        
        # if we don't do this, pytorch will accumulate gradients (summation) from previous backprop to next backprop and so on...
        # this behaviour is useful when we can't fit many samples in memory but we need to compute gradient on large batch. In this case, we simply accumulate gradient and do backprop
        # after enough samples has been seen.
        optimizer.zero_grad()
        #forward
        output = pred_model(x)
        # print(output)
        # print(output.long())
        # print(y.shape)
        loss = criterion(output.squeeze(0), y.squeeze(1))
        # print(output.loss.item())
        #calculate accuracy
        # acc = calculate_accuracy(log, y)

        #Back propagation
        loss.backward()
        
        #Gradient step
        optimizer.step()

        epoch_loss += loss.item()
        # epoch_acc += acc.item()
        
        if (i + 1) % 200 == 0:
            dev_loss = 0 #valid_loss(pred_model, validation)
            if validation_loss is None or validation_loss > dev_loss:
                validation_loss = dev_loss
                # Save best model
                # torch.save(pred_model.state_dict(), '/content/gdrive/My Drive/NL3.14/resources/prediction_model') 
            print(f'Epoch {epoch} batch {i} | Avg Train Loss: {epoch_loss/(i + 1):.6f} | Current validation Loss {dev_loss:.6f}| Minimal validation Loss {validation_loss:.6f} | Current Perplexity {torch.exp(torch.tensor(dev_loss))} | Time passed {datetime.now() - t}')
        i+=1
    print(f'Epoch {epoch} | Avg Train Loss: {epoch_loss/batches:.6f} | Minimal validation Loss {validation_loss:.6f}')  

4157 batches
Epoch 1 batch 199 | Avg Train Loss: 0.333203 | Current validation Loss 0.196487| Minimal validation Loss 0.196487 | Current Perplexity 1.2171193361282349 | Time passed 0:01:26.556693
Epoch 1 batch 399 | Avg Train Loss: 0.172870 | Current validation Loss 0.053391| Minimal validation Loss 0.053391 | Current Perplexity 1.0548418760299683 | Time passed 0:02:53.646495
Epoch 1 batch 599 | Avg Train Loss: 0.116824 | Current validation Loss 0.026861| Minimal validation Loss 0.026861 | Current Perplexity 1.0272254943847656 | Time passed 0:04:21.067256
Epoch 1 batch 799 | Avg Train Loss: 0.088285 | Current validation Loss 0.016972| Minimal validation Loss 0.016972 | Current Perplexity 1.0171165466308594 | Time passed 0:05:48.070087
Epoch 1 batch 999 | Avg Train Loss: 0.070985 | Current validation Loss 0.011909| Minimal validation Loss 0.011909 | Current Perplexity 1.011980414390564 | Time passed 0:07:15.232507
Epoch 1 batch 1199 | Avg Train Loss: 0.059370 | Current validation Loss 0

KeyboardInterrupt: ignored